In [ ]:
]st

In [ ]:
using Revise

In [ ]:
using JuliaDB, TaylorSeries, PlanetaryEphemeris

In [ ]:
using JLD

In [ ]:
using Apophis

In [ ]:
using Statistics

In [ ]:
using LinearAlgebra

In [ ]:
using Dates

In [ ]:
using Plots

In [ ]:
dq = set_variables("δx", order=5, numvars=7)

In [ ]:
Apophis.loadjpleph()

In [ ]:
# compute debiasing corrections with 2014 table
radec_corr_0408_f15 = radec_table("../data/99942_2004_2008.dat", debias_table="2014")
ras_corr_0408_f15 = select(radec_corr_0408_f15, :α_corr)
dec_corr_0408_f15 = select(radec_corr_0408_f15, :δ_corr)
radec_corr_1120_f15 = radec_table("../data/99942_2011_2020.dat", debias_table="2014")
ras_corr_1120_f15 = select(radec_corr_1120_f15, :α_corr)
dec_corr_1120_f15 = select(radec_corr_1120_f15, :δ_corr)
;

In [ ]:
_radec_0408 = JuliaDB.load("radec_99942_2004_2008.jdb")
radec_0408 = insertcolsafter(_radec_0408, length(columns(_radec_0408)), :α_corr_2014 => ras_corr_0408_f15, :δ_corr_2014 => dec_corr_0408_f15);

In [ ]:
_radec_1120 = JuliaDB.load("radec_99942_2011_2020.jdb")
radec_1120 = insertcolsafter(_radec_1120, length(columns(_radec_1120)), :α_corr_2014 => ras_corr_1120_f15, :δ_corr_2014 => dec_corr_1120_f15);

In [ ]:
del_comp_0506 = JLD.load("deldop_m5y_o5v7_BUG_FIXED_MAYBE.jld", "vdel")
dop_comp_0506 = JLD.load("deldop_m5y_o5v7_BUG_FIXED_MAYBE.jld", "vdop");

In [ ]:
del_comp_1213 = JLD.load("deldop_p6y_o5v7_BUG_FIXED_MAYBE.jld", "vdel")
dop_comp_1213 = JLD.load("deldop_p6y_o5v7_BUG_FIXED_MAYBE.jld", "vdop");

In [ ]:
deldop_obs_0506 = process_radar_data_jpl("../data/Apophis_JPL_data_2005_2006.dat")
deldop_obs_1213 = process_radar_data_jpl("../data/Apophis_JPL_data_2012_2013.dat");

In [ ]:
del_ind_0506 = findall(x->x.delay_units == "us", deldop_obs_0506)
dop_ind_0506 = findall(x->x.doppler_units == "Hz", deldop_obs_0506)
del_ind_1213 = findall(x->x.delay_units == "us", deldop_obs_1213)
dop_ind_1213 = findall(x->x.doppler_units == "Hz", deldop_obs_1213)
;

In [ ]:
del_obs_0506 = delay.(deldop_obs_0506[del_ind_0506])
dop_obs_0506 = doppler.(deldop_obs_0506[dop_ind_0506])
del_obs_1213 = delay.(deldop_obs_1213[del_ind_1213])
dop_obs_1213 = doppler.(deldop_obs_1213[dop_ind_1213])
;

In [ ]:
σ_del_0506

In [ ]:
σ_del_0506 = delay_sigma.(deldop_obs_0506[del_ind_0506])
σ_dop_0506 = doppler_sigma.(deldop_obs_0506[dop_ind_0506])
σ_del_1213 = delay_sigma.(deldop_obs_1213[del_ind_1213])
σ_dop_1213 = doppler_sigma.(deldop_obs_1213[dop_ind_1213])
;

In [ ]:
del_0506 = table((τ_obs=del_obs_0506,τ_comp=del_comp_0506,σ_τ=σ_del_0506))
del_1213 = table((τ_obs=del_obs_1213,τ_comp=del_comp_1213,σ_τ=σ_del_1213))
dop_0506 = table((ν_obs=dop_obs_0506,ν_comp=dop_comp_0506,σ_ν=σ_dop_0506))
dop_1213 = table((ν_obs=dop_obs_1213,ν_comp=dop_comp_1213,σ_ν=σ_dop_1213));

In [ ]:
del = merge(del_0506, del_1213)

In [ ]:
dop = merge(dop_0506, dop_1213)

In [ ]:
res_τ = select(del, (:τ_obs, :τ_comp)=>x->x[1]-x[2])
res_ν = select(dop, (:ν_obs, :ν_comp)=>x->x[1]-x[2])

w_τ = select(del, :σ_τ=>x->1/x^2)
w_ν = select(dop, :σ_ν=>x->1/x^2)
;

In [ ]:
radec_0415 = merge(radec_0408, filter(x -> x.yr<=2015, radec_1120))
radec_0420 = merge(radec_0408, radec_1120)
;

In [ ]:
length(radec_0415), length(radec_0420)

In [ ]:
# check if all RA/Dec observations are CCD:

all(select(radec_0420, :j2000=>x->x == "C" || x == "X"))

In [ ]:
#filter(x->x.catalog == " ", radec_0420) # there are 86 RA/Dec obs without catalog info

In [ ]:
#radec_04XX = radec_0415
#radec_04XX = radec_0420

χ2_rej = 4.25 # Brozovic et al (2018) use Chesley et al (2010) outlier rejection, in turn from Carpino et al (2003)
radec_04XX = filter(x->((x.α_obs-x.α_corr_2014-x.α_comp())^2 + (x.δ_obs-x.δ_corr_2014-x.δ_comp())^2)<=χ2_rej, radec_0415)
#radec_04XX = filter(x->((x.α_obs-x.α_corr-x.α_comp())^2 + (x.δ_obs-x.δ_corr-x.δ_comp())^2)<χ2_rej, radec_0420)

In [ ]:
res_α_2014 = select(radec_04XX, (:α_obs, :α_corr_2014, :α_comp)=>x->(x[1]-x[2]-x[3]))
res_δ_2014 = select(radec_04XX, (:δ_obs, :δ_corr_2014, :δ_comp)=>x->(x[1]-x[2]-x[3]))
res_α_2018 = select(radec_04XX, (:α_obs, :α_corr, :α_comp)=>x->(x[1]-x[2]-x[3]))
res_δ_2018 = select(radec_04XX, (:δ_obs, :δ_corr, :δ_comp)=>x->(x[1]-x[2]-x[3]))
res2_α_2014 = res_α_2014.^2
res2_δ_2014 = res_δ_2014.^2
res2_α_2018 = res_α_2018.^2
res2_δ_2018 = res_δ_2018.^2
;

In [ ]:
# for each observation bath, count the number of observations made in the same night (?) by the same observatory
date_obscode_v = select(radec_04XX, (:dt_utc_obs,:obscode)=>x->(Date(x[1]),x[2]))
Nv = [count(x->x[1]==i[1] && x[2]==i[2], date_obscode_v) for i in date_obscode_v]
relax_factor = map(x->x>4 ? sqrt(x/4) : 1.0, Nv);

# statistical weights according to Veres et al (2017)
σ_v17 = map(x->weights_veres2017(x), radec_04XX).*(relax_factor)

w_α = 1 ./ (σ_v17.^2)
w_δ = 1 ./ (σ_v17.^2)
;

In [ ]:
length(w_τ), length(w_ν), length(w_α), length(w_δ)

In [ ]:
length(del), length(dop), length(res_α_2014), length(res_δ_2014), length(res_α_2018), length(res_δ_2018)

In [ ]:
nobs_R = length(del) + length(dop)

In [ ]:
nobs_O = 2length(radec_04XX)

In [ ]:
nobs = nobs_R + nobs_O

In [ ]:
res_O_2014 = vcat(res_α_2014, res_δ_2014)
res_O_2018 = vcat(res_α_2018, res_δ_2018)
w_O = vcat(w_α, w_δ)

res_R = vcat(res_τ, res_ν)
w_R = vcat(w_τ, w_ν)

res_OR_2014 = vcat(res_R, res_O_2014)
res_OR_2018 = vcat(res_R, res_O_2018)
w_OR = vcat(w_R, w_O)
;

In [ ]:
x_OR7_2014, Γ_OR7_2014 = newtonls(res_OR_2014, w_OR, zeros(7))
σ_OR7_2014 = sqrt.(diag(Γ_OR7_2014))
@show x_OR7_2014 σ_OR7_2014 x_OR7_2014[end]/σ_OR7_2014[end]

x_OR7_2018, Γ_OR7_2018 = newtonls(res_OR_2018, w_OR, zeros(7))
σ_OR7_2018 = sqrt.(diag(Γ_OR7_2018))
@show x_OR7_2018 σ_OR7_2018 x_OR7_2018[end]/σ_OR7_2018[end]

x_OR6_2014, Γ_OR6_2014 = newtonls_6v(res_OR_2014, w_OR, zeros(7))
σ_OR6_2014 = sqrt.(diag(Γ_OR6_2014))
@show x_OR6_2014 σ_OR6_2014 x_OR6_2014[end]/σ_OR6_2014[end]

x_OR6_2018, Γ_OR6_2018 = newtonls_6v(res_OR_2018, w_OR, zeros(7))
σ_OR6_2018 = sqrt.(diag(Γ_OR6_2018))
@show x_OR6_2018 σ_OR6_2018 x_OR6_2018[end]/σ_OR6_2018[end]

x_O7_2014, Γ_O7_2014 = newtonls(res_O_2014, w_O, zeros(7))
σ_O7_2014 = sqrt.(diag(Γ_O7_2014))
@show x_O7_2014 σ_O7_2014 x_O7_2014[end]/σ_O7_2014[end]

x_O7_2018, Γ_O7_2018 = newtonls(res_O_2018, w_O, zeros(7))
σ_O7_2018 = sqrt.(diag(Γ_O7_2018))
@show x_O7_2018 σ_O7_2018 x_O7_2018[end]/σ_O7_2018[end]

x_R7, Γ_R7 = newtonls(res_R, w_R, vcat(zeros(6),10))#zeros(7))
σ_R7 = sqrt.(diag(Γ_R7))
@show x_R7 σ_R7 x_R7[end]/σ_R7[end]
;

In [ ]:
x_OR7_2014[end]/σ_OR7_2014[end], x_OR7_2018[end]/σ_OR7_2018[end]

In [ ]:
#Q_del = sum( select(del, (:τ_obs, :τ_comp, :σ_τ)=>x->((x[1]-x[2])/x[3])^2) )
#Q_dop = sum( select(dop, (:ν_obs, :ν_comp, :σ_ν)=>x->((x[1]-x[2])/x[3])^2) )
#Q_deldop = Q_del + Q_dop;

#Q_radec = sum( vcat(w_α,w_δ).*vcat(res2_α,res2_δ) )
#Q = (Q_deldop + Q_radec)/(nobs_O + nobs_R);
;

In [ ]:
res_α_OR7_2014 = res_α_2014(x_OR7_2014)
res_δ_OR7_2014 = res_δ_2014(x_OR7_2014)
res_α_OR7_2018 = res_α_2018(x_OR7_2018)
res_δ_OR7_2018 = res_δ_2018(x_OR7_2018)
;

In [ ]:
mean(res_α_OR7_2014), std(res_α_OR7_2014), mean(res_δ_OR7_2014), std(res_δ_OR7_2014)

In [ ]:
mean(res_α_OR7_2018), std(res_α_OR7_2018), mean(res_δ_OR7_2018), std(res_δ_OR7_2018)

In [ ]:
minimum(res_α_OR7_2014), maximum(res_α_OR7_2014)

In [ ]:
minimum(res_δ_OR7_2014), maximum(res_δ_OR7_2014)

In [ ]:
gr()

In [ ]:
histogram(res_α_OR7_2014)
histogram!(res_α_OR7_2018)

In [ ]:
histogram(res_δ_OR7_2014)
histogram!(res_δ_OR7_2018)

In [ ]:
scatter(res_α_OR7_2014, res_δ_OR7_2014, ms=1.0, m=:circle, msc=:auto)
scatter!(res_α_OR7_2018, res_δ_OR7_2018, ms=1.0, m=:square, msc=:auto)

In [ ]:
scatter(res_τ(x_OR7_2014), yerror=vcat(σ_del_0506, σ_del_1213))
scatter!(res_τ(x_OR7_2018), yerror=vcat(σ_del_0506, σ_del_1213))

In [ ]:
scatter(res_ν(x_OR7_2014), yerror=vcat(σ_dop_0506, σ_dop_1213))
scatter!(res_ν(x_OR7_2018), yerror=vcat(σ_dop_0506, σ_dop_1213))

In [ ]:
radec_19_20 = filter(x->x.yr>2019, radec_1120)
t_19_20 = 2000 .+ (select(radec_19_20, :dt_utc_obs=>x->datetime2julian(x)) .- J2000) ./ yr
α_19_20_obs = select(radec_19_20, (:α_obs, :α_corr_2014)=>x->x[1]-x[2])
δ_19_20_obs = select(radec_19_20, (:δ_obs, :δ_corr_2014)=>x->x[1]-x[2])
α_19_20_OR6 = select(radec_19_20, :α_comp)(x_OR6_2014)
δ_19_20_OR6 = select(radec_19_20, :δ_comp)(x_OR6_2014)
α_19_20_OR7 = select(radec_19_20, :α_comp)(x_OR7_2014)
δ_19_20_OR7 = select(radec_19_20, :δ_comp)(x_OR7_2014)
res_α_19_20_obs_OR6 = α_19_20_obs .- α_19_20_OR6
res_δ_19_20_obs_OR6 = δ_19_20_obs .- δ_19_20_OR6
res_α_19_20_obs_OR7 = α_19_20_obs .- α_19_20_OR7
res_δ_19_20_obs_OR7 = δ_19_20_obs .- δ_19_20_OR7
;

In [ ]:
angv = vcat(range(0, length=1000, stop=prevfloat(2pi)), 0)
sigma_factor = 3.0
σ_cos_ang = sigma_factor*cos.(angv)
σ_sin_ang = sigma_factor*sin.(angv);

In [ ]:
cov_αδ_cholf_OR6 = cholesky(cov(hcat(res_α_19_20_obs_OR6, res_δ_19_20_obs_OR6)))

In [ ]:
cov_αδ_cholf_OR7 = cholesky(cov(hcat(res_α_19_20_obs_OR7, res_δ_19_20_obs_OR7)))

In [ ]:
ell_3σ_x_OR6 = cov_αδ_cholf_OR6.L[1,1].*σ_cos_ang .+ cov_αδ_cholf_OR6.L[1,2].*σ_sin_ang
ell_3σ_y_OR6 = cov_αδ_cholf_OR6.L[2,1].*σ_cos_ang .+ cov_αδ_cholf_OR6.L[2,2].*σ_sin_ang

ell_3σ_x_OR7 = cov_αδ_cholf_OR7.L[1,1].*σ_cos_ang .+ cov_αδ_cholf_OR7.L[1,2].*σ_sin_ang
ell_3σ_y_OR7 = cov_αδ_cholf_OR7.L[2,1].*σ_cos_ang .+ cov_αδ_cholf_OR7.L[2,2].*σ_sin_ang
;

In [ ]:
scatter((t_19_20.-2020)*12, res_α_19_20_obs_OR6, msc=:auto, m=:xcross, ms=3.0, label="OR6")
scatter!((t_19_20.-2020)*12, res_α_19_20_obs_OR7, msc=:auto, m=:square, ms=3.0, label="OR7")
xlabel!("2020 month")
ylabel!("αcos(δ) [arcseconds]")

In [ ]:
scatter((t_19_20.-2020)*12, res_δ_19_20_obs_OR6, msc=:auto, m=:xcross, ms=3.0, label="OR6")
scatter!((t_19_20.-2020)*12, res_δ_19_20_obs_OR7, msc=:auto, m=:square, ms=3.0, label="OR7")
xlabel!("2020 month")
ylabel!("δ [arcseconds]")

In [ ]:
scatter(res_α_19_20_obs_OR6, res_δ_19_20_obs_OR6, msc=:auto, m=:xcross, ms=3.0, label="OR6")
scatter!(res_α_19_20_obs_OR7, res_δ_19_20_obs_OR7, msc=:auto, m=:square, ms=2.0, label="OR7")
plot!(ell_3σ_x_OR6, ell_3σ_y_OR6, color=1)
plot!(ell_3σ_x_OR7, ell_3σ_y_OR7, color=2)
xlabel!("αcos(δ) [arcsec]")
ylabel!("δ [arcsec]")